# Part-of-Speech Tagging

- using LSTM
- Very simple example to understand concept

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [0]:
# For data processing and maths
import numpy as np
import pandas as pd
import time
import math
import os
#For Visuals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [0]:
# for deep learning and neural network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
! pip install version_information

In [5]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,torch,numpy,seaborn, matplotlib

Software versions
Python 3.6.8 64bit [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.24.2
torch 1.3.0+cu100
numpy 1.16.5
seaborn 0.9.0
matplotlib 3.0.3
Tue Oct 22 15:17:35 2019 UTC

# 2)-  Loading data & its preprocessing

In [6]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [7]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [0]:
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')

In [9]:
# Let's have a look at some sentences

print(tagged_sentence[3])

[('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('cigarette', 'NOUN'), ('filters', 'NOUN'), ('has', 'VERB'), ('caused', 'VERB'), ('a', 'DET'), ('high', 'ADJ'), ('percentage', 'NOUN'), ('of', 'ADP'), ('cancer', 'NOUN'), ('deaths', 'NOUN'), ('among', 'ADP'), ('a', 'DET'), ('group', 'NOUN'), ('of', 'ADP'), ('workers', 'NOUN'), ('exposed', 'VERB'), ('*', 'X'), ('to', 'PRT'), ('it', 'PRON'), ('more', 'ADV'), ('than', 'ADP'), ('30', 'NUM'), ('years', 'NOUN'), ('ago', 'ADP'), (',', '.'), ('researchers', 'NOUN'), ('reported', 'VERB'), ('0', 'X'), ('*T*-1', 'X'), ('.', '.')]


In [10]:
print("Number of Tagged Sentences ",len(tagged_sentence))

Number of Tagged Sentences  3914


### 2a)- Preprocess Data

- A word/tag dictionary,
- A letter/character dictionary
- A POS tag dictionary

In [0]:
def word_to_ix(word, ix):
    return torch.tensor(ix[word], dtype = torch.long)

In [0]:
def char_to_ix(char, ix):
    return torch.tensor(ix[char], dtype= torch.long)


In [0]:
def tag_to_ix(tag, ix):
    return torch.tensor(ix[tag], dtype= torch.long)

In [0]:
def sequence_to_idx(sequence, ix):
    return torch.tensor([ix[s] for s in sequence], dtype=torch.long)

In [0]:
word_to_idx = {}
tag_to_idx = {}
char_to_idx = {}
for sentence in tagged_sentence:
    for word, pos_tag in sentence:
        if word not in word_to_idx.keys():
            word_to_idx[word] = len(word_to_idx)
        if pos_tag not in tag_to_idx.keys():
            tag_to_idx[pos_tag] = len(tag_to_idx)
        for char in word:
            if char not in char_to_idx.keys():
                char_to_idx[char] = len(char_to_idx)

In [16]:
print(type(word_to_idx))
print(type(tag_to_idx))
print(type(char_to_idx))

<class 'dict'>
<class 'dict'>
<class 'dict'>


In [17]:
pd.DataFrame(word_to_idx, index=[0])

,Pierre,Vinken,",",61,years,old,will,join,the,board,as,a,nonexecutive,director,Nov.,29,.,Mr.,is,chairman,of,Elsevier,N.V.,Dutch,publishing,group,Rudolph,Agnew,55,and,former,Consolidated,Gold,Fields,PLC,was,named,*-1,this,British,...,TXO,Proceeds,15.5,257,Icahn,13.1,721,2.62,598,2.07,Barrett,Leon,Level,McFarlan,DiLoreto,container,Delmont,constitutional-law,Professors,Kurland,Laurence,Tribe,procedure,scuttle,implicitly,Kennedy,professors,spectrum,authorizes,partial,Civil,shared,lawmaking,supports,reckless,Trinity,railcar,platforms,Trailer,Train
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,12368,12369,12370,12371,12372,12373,12374,12375,12376,12377,12378,12379,12380,12381,12382,12383,12384,12385,12386,12387,12388,12389,12390,12391,12392,12393,12394,12395,12396,12397,12398,12399,12400,12401,12402,12403,12404,12405,12406,12407


In [18]:
pd.DataFrame(tag_to_idx, index=[0])

,NOUN,.,NUM,ADJ,VERB,DET,ADP,CONJ,X,ADV,PRT,PRON
0,0,1,2,3,4,5,6,7,8,9,10,11


These are our our 12 categories for part of speech

In [19]:
pd.DataFrame(char_to_idx, index=[0])

,P,i,e,r,V,n,k,",",6,1,y,a,s,o,l,d,w,j,t,h,b,x,c,u,v,N,.,2,9,M,m,f,E,D,p,g,R,A,5,C,G,F,L,*,-,B,K,3,0,T,I,Y,',J,`,W,q,H,U,8,4,?,z,&,S,7,%,$,;,O,:,Q,Z,\,/,X,@,!,#
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78


In [0]:
word_vocab_size = len(word_to_idx)
tag_vocab_size = len(tag_to_idx)
char_vocab_size = len(char_to_idx)

In [21]:
print("Unique words: {}".format(len(word_to_idx)))
print("Unique tags: {}".format(len(tag_to_idx)))
print("Unique characters: {}".format(len(char_to_idx)))

Unique words: 12408
Unique tags: 12
Unique characters: 79


### 2b)-Train/Test Splitting the data

In [0]:
from sklearn.model_selection import train_test_split
Train,Test = train_test_split(tagged_sentence, test_size=0.20, random_state=42)

In [23]:
Train[:3]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('And', 'CONJ'),
  ('it', 'PRON'),
  ('was', 'VERB'),
  ('stupid', 'ADJ'),
  ('.', '.')],
 [('Some', 'DET'),
  ('dealers', 'NOUN'),
  ('said', 'VERB'),
  ('0', 'X'),
  ('the', 'DET'),
  ('dollar', 'NOUN'),
  ('was', 'VERB'),
  ('pressured', 'ADJ'),
  ('*-1', 'X'),
  ('slightly', 'ADV'),
  ('because', 'ADP'),
  ('a', 'DET'),
  ('number', 'NOUN'),
  ('of', 'ADP'),
  ('market', 'NOUN'),
  ('participants', 'NOUN'),
  ('had', 'VERB'),
  ('boosted', 'VERB'),
  ('their', 'PRON'),
  ('expectations', 'NOUN'),
  ('in', 'ADP'),
  ('the', 'DET'),
  ('past', 'ADJ'),
  ('day', 'NOUN'),
  ('and', 'CONJ'),
  ('were', 'VERB'),
  ('looking', 'VERB'),
  ('for', 'ADP

In [24]:
Test[:3]

[[('For', 'ADP'),
  ('the', 'DET'),
  ('Agency', 'NOUN'),
  ('for', 'ADP'),
  ('International', 'NOUN'),
  ('Development', 'NOUN'),
  (',', '.'),
  ('appropriators', 'NOUN'),
  ('approved', 'VERB'),
  ('$', '.'),
  ('200', 'NUM'),
  ('million', 'NUM'),
  ('*U*', 'X'),
  ('in', 'ADP'),
  ('secondary', 'ADJ'),
  ('loan', 'NOUN'),
  ('guarantees', 'NOUN'),
  ('under', 'ADP'),
  ('an', 'DET'),
  ('expanded', 'VERB'),
  ('trade', 'NOUN'),
  ('credit', 'NOUN'),
  ('insurance', 'NOUN'),
  ('program', 'NOUN'),
  (',', '.'),
  ('and', 'CONJ'),
  ('total', 'ADJ'),
  ('loan', 'NOUN'),
  ('guarantees', 'NOUN'),
  ('for', 'ADP'),
  ('the', 'DET'),
  ('Overseas', 'NOUN'),
  ('Private', 'NOUN'),
  ('Investment', 'NOUN'),
  ('Corp.', 'NOUN'),
  ('are', 'VERB'),
  ('increased', 'VERB'),
  ('*-3', 'X'),
  ('by', 'ADP'),
  ('$', '.'),
  ('40', 'NUM'),
  ('million', 'NUM'),
  ('*U*', 'X'),
  ('over', 'ADP'),
  ('fiscal', 'ADJ'),
  ('1989', 'NUM'),
  ('as', 'ADP'),
  ('part', 'NOUN'),
  ('of', 'ADP'),
  ('

**Alternative way**

In [0]:
import random

tr_random = random.sample(list(range(len(tagged_sentence))), int(0.95 * len(tagged_sentence)))

train = [tagged_sentence[i] for i in tr_random]
test = [tagged_sentence[i] for i in range(len(tagged_sentence)) if i not in tr_random]

In [26]:
train[:3]

[[('``', '.'),
  ('Only', 'ADV'),
  ('five', 'NUM'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('40', 'NUM'),
  ('questions', 'NOUN'),
  ('were', 'VERB'),
  ('geography', 'NOUN'),
  ('questions', 'NOUN'),
  ('.', '.')],
 [('Williams', 'NOUN'),
  (',', '.'),
  ('Duluth', 'NOUN'),
  (',', '.'),
  ('Ga.', 'NOUN'),
  (',', '.'),
  ('is', 'VERB'),
  ('an', 'DET'),
  ('insurance', 'NOUN'),
  ('and', 'CONJ'),
  ('financial-services', 'NOUN'),
  ('holding', 'VERB'),
  ('company', 'NOUN'),
  ('.', '.')],
 [('Employers', 'NOUN'),
  ('can', 'VERB'),
  ('pay', 'VERB'),
  ('the', 'DET'),
  ('subminimum', 'NOUN'),
  ('for', 'ADP'),
  ('90', 'NUM'),
  ('days', 'NOUN'),
  (',', '.'),
  ('without', 'ADP'),
  ('restriction', 'NOUN'),
  (',', '.'),
  ('to', 'PRT'),
  ('workers', 'NOUN'),
  ('with', 'ADP'),
  ('less', 'ADJ'),
  ('than', 'ADP'),
  ('six', 'NUM'),
  ('months', 'NOUN'),
  ('of', 'ADP'),
  ('job', 'NOUN'),
  ('experience', 'NOUN'),
  ('*NOT*', 'X'),
  (',', '.'),
  ('and', 'CONJ'),
  ('for', 'ADP'

In [27]:
test[:3]

[[('Four', 'NUM'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('five', 'NUM'),
  ('surviving', 'VERB'),
  ('workers', 'NOUN'),
  ('have', 'VERB'),
  ('asbestos-related', 'ADJ'),
  ('diseases', 'NOUN'),
  (',', '.'),
  ('including', 'VERB'),
  ('three', 'NUM'),
  ('with', 'ADP'),
  ('recently', 'ADV'),
  ('diagnosed', 'VERB'),
  ('cancer', 'NOUN'),
  ('.', '.')],
 [('The', 'DET'),
  ('30-day', 'ADJ'),
  ('simple', 'ADJ'),
  ('yield', 'NOUN'),
  ('fell', 'VERB'),
  ('to', 'PRT'),
  ('an', 'DET'),
  ('average', 'ADJ'),
  ('8.19', 'NUM'),
  ('%', 'NOUN'),
  ('from', 'ADP'),
  ('8.22', 'NUM'),
  ('%', 'NOUN'),
  (';', '.'),
  ('the', 'DET'),
  ('30-day', 'ADJ'),
  ('compound', 'NOUN'),
  ('yield', 'NOUN'),
  ('slid', 'VERB'),
  ('to', 'PRT'),
  ('an', 'DET'),
  ('average', 'ADJ'),
  ('8.53', 'NUM'),
  ('%', 'NOUN'),
  ('from', 'ADP'),
  ('8.56', 'NUM'),
  ('%', 'NOUN'),
  ('.', '.')],
 [('Finmeccanica', 'NOUN'),
  ('is', 'VERB'),
  ('an', 'DET'),
  ('Italian', 'ADJ'),
  ('state-owned', 'ADJ'),
  

# 3)- Model Building

### 3.1)-Model Parameters

In [0]:
WORD_EMBEDDING_DIM = 1024
CHAR_EMBEDDING_DIM = 128
WORD_HIDDEN_DIM = 1024
CHAR_HIDDEN_DIM = 1024
EPOCHS = 70

### 3.2)-Model Classes


I will construct the model classes, the Word LSTM tagger and the Character LSTM Tagger. The LSTM in both cases takes in a sequence (words or characters), embeds the sequence into an embedding space (dimension of the space is a hyperparameter), and runs the LSTM model.

In our case, first the word level LSTM will take in a sequence of words and convert them into the word embedding space. Similarly, it will take the words sequentially and run the character level LSTM model, which will first take in the sequence of characters in each word and project it in the character embedding space and then run the LSTM model and take its hidden state and feed it back to the word LSTM model.

Using the character level hidden representation for every word as well as the word embedding, the word level model then runs LSTM on the sequence of words, and outputs the predictions for every tag. This prediction is later processed to find the corresponding tags.

- Credit: https://www.kaggle.com/krishanudb/lstm-character-word-pos-tag-model-pytorch/data

In [0]:
class DualLSTMTagger(nn.Module):
    def __init__(self, word_embedding_dim, word_hidden_dim, char_embedding_dim, char_hidden_dim, word_vocab_size, char_vocab_size, tag_vocab_size):
        super(DualLSTMTagger, self).__init__()
        self.word_embedding = nn.Embedding(word_vocab_size, word_embedding_dim)
        
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim, char_hidden_dim)
        
        self.lstm = nn.LSTM(word_embedding_dim + char_hidden_dim, word_hidden_dim)
        self.hidden2tag = nn.Linear(word_hidden_dim, tag_vocab_size)
        
    def forward(self, sentence, words):
        embeds = self.word_embedding(sentence)
        char_hidden_final = []
        for word in words:
            char_embeds = self.char_embedding(word)
            _, (char_hidden, char_cell_state) = self.char_lstm(char_embeds.view(len(word), 1, -1))
            word_char_hidden_state = char_hidden.view(-1)
            char_hidden_final.append(word_char_hidden_state)
        char_hidden_final = torch.stack(tuple(char_hidden_final))
        
        combined = torch.cat((embeds, char_hidden_final), 1)

        lstm_out, _ = self.lstm(combined.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

**training is too computing intensive. But, concept and code has been provided**

In [0]:
model = DualLSTMTagger(WORD_EMBEDDING_DIM, WORD_HIDDEN_DIM, CHAR_EMBEDDING_DIM, CHAR_HIDDEN_DIM, word_vocab_size, char_vocab_size, tag_vocab_size)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

if use_cuda:
    model.cuda()

# Define the loss function as the Negative Log Likelihood loss (NLLLoss)
loss_function = nn.NLLLoss()

# We will be using a simple SGD optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# The test sentence
seq = "everybody eat the food . I kept looking out the window , trying to find the one I was waiting for .".split()
print("Running a check on the model before training.\nSentences:\n{}".format(" ".join(seq)))
with torch.no_grad():
    words = [torch.tensor(sequence_to_idx(s[0], char_to_idx), dtype=torch.long).to(device) for s in seq]
    sentence = torch.tensor(sequence_to_idx(seq, word_to_idx), dtype=torch.long).to(device)
        
    tag_scores = model(sentence, words)
    _, indices = torch.max(tag_scores, 1)
    ret = []
    for i in range(len(indices)):
        for key, value in tag_to_idx.items():
            if indices[i] == value:
                ret.append((seq[i], key))
    print(ret)
# Training start
print("Training Started")
accuracy_list = []
loss_list = []
interval = round(len(train) / 100.)
epochs = EPOCHS
e_interval = round(epochs / 10.)
for epoch in range(epochs):
    acc = 0 #to keep track of accuracy
    loss = 0 # To keep track of the loss value
    i = 0
    for sentence_tag in train:
        i += 1
        words = [torch.tensor(sequence_to_idx(s[0], char_to_idx), dtype=torch.long).to(device) for s in sentence_tag]
        sentence = [s[0] for s in sentence_tag]
        sentence = torch.tensor(sequence_to_idx(sentence, word_to_idx), dtype=torch.long).to(device)
        targets = [s[1] for s in sentence_tag]
        targets = torch.tensor(sequence_to_idx(targets, tag_to_idx), dtype=torch.long).to(device)
        
        model.zero_grad()
        
        tag_scores = model(sentence, words)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        loss += loss.item()
        _, indices = torch.max(tag_scores, 1)
#         print(indices == targets)
        acc += torch.mean(torch.tensor(targets == indices, dtype=torch.float))
        if i % interval == 0:
            print("Epoch {} Running;\t{}% Complete".format(epoch + 1, i / interval), end = "\r", flush = True)
    loss = loss / len(train)
    acc = acc / len(train)
    loss_list.append(float(loss))
    accuracy_list.append(float(acc))
    if (epoch + 1) % e_interval == 0:
        print("Epoch {} Completed,\tLoss {}\tAccuracy: {}".format(epoch + 1, np.mean(loss_list[-e_interval:]), np.mean(accuracy_list[-e_interval:])))

Running a check on the model before training.
Sentences:
everybody eat the food . I kept looking out the window , trying to find the one I was waiting for .
[('everybody', 'ADV'), ('eat', 'NUM'), ('the', 'ADV'), ('food', 'ADJ'), ('.', 'X'), ('I', 'ADV'), ('kept', 'ADV'), ('looking', 'ADV'), ('out', 'ADV'), ('the', 'ADV'), ('window', 'ADV'), (',', 'ADV'), ('trying', 'ADV'), ('to', 'ADV'), ('find', 'ADV'), ('the', 'ADV'), ('one', 'ADV'), ('I', 'ADV'), ('was', '.'), ('waiting', 'ADJ'), ('for', 'ADV'), ('.', 'VERB')]
Training Started


# 4)-Plotting results

In [0]:
import matplotlib.pyplot as plt
plt.plot(accuracy_list, c="red", label ="Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend()
plt.show()

plt.plot(loss_list, c="blue", label ="Loss")
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend()
plt.show()

# 5)- Testing results

In [0]:
with torch.no_grad():
    words = [torch.tensor(sequence_to_idx(s[0], char_to_idx), dtype=torch.long).to(device) for s in seq]
    sentence = torch.tensor(sequence_to_idx(seq, word_to_idx), dtype=torch.long).to(device)
        
    tag_scores = model(sentence, words)
    _, indices = torch.max(tag_scores, 1)
    ret = []
    for i in range(len(indices)):
        for key, value in tag_to_idx.items():
            if indices[i] == value:
                ret.append((seq[i], key))
    print(ret)